**OTUS Machine Learning Professional**

### **Homework 7**

### Building a recommendation system

![hw67_header](https://user-images.githubusercontent.com/73858914/171476388-329bdb16-898d-46c4-ba66-6233dabb7244.png)

**Goals:**

1. Download and clean "Metropolitan Interstate Traffic Volume" dataset;   
2. Explore data;
3. Engineer additional features;
4. Build a baseline model - average by day of week and hour;
5. Build and test a number of models and compare results with the baseline;
6. Make predictions of temperature for test set and use it in model;
7. Visualize best model with confidence intervals.

**Means**:

- All time series exploration and prognosis are done with [Pycaret-ts-alpha](https://pypi.org/project/pycaret-ts-alpha) library

**Data source**:

[UCI Machine Learning Repository: Metro Interstate Traffic Volume Data Set](https://archive.ics.uci.edu/ml/datasets/Metro+Interstate+Traffic+Volume)

**Binder notebook:**

main.ipynb

### Import libraries

In [9]:
import pandas as pd
import numpy as np
import gzip
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import warnings

warnings.filterwarnings("ignore")

###  Miscellaneous settings

In [10]:
%matplotlib inline
display(HTML("<style>.container { width:80% !important; }</style>"))
%config InlineBackend.figure_format = 'retina'
plt.rcParams["figure.figsize"] = (12, 4)
plt.style.use("ggplot")
pd.options.display.precision = 4

### Load data

In [11]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('./data/Software_5.json.gz')

In [12]:
df.tail()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
12800,4.0,False,"07 16, 2016",A1E50L7PCVXLN4,B01FFVDY9M,{'Platform:': ' Key Card'},Colinda,When I ordered this it was listed as Photo Edi...,File Management Software with Basic Editing Ca...,1468627200,NaN,NaN
12801,3.0,False,"06 17, 2017",AVU1ILDDYW301,B01HAP3NUG,NaN,G. Hearn,This software has SO much going on. Theres a ...,"Might not be for the ""novice""",1497657600,NaN,NaN
12802,4.0,False,"01 24, 2017",A2LW5AL0KQ9P1M,B01HAP3NUG,NaN,Dr. E,I have used both more complex and less complex...,"Great, Inexpensive Software for Those Who Have...",1485216000,NaN,NaN
12803,3.0,False,"06 14, 2018",AZ515FFZ7I2P7,B01HAP47PQ,{'Platform:': ' PC Disc'},Jerry Jackson Jr.,Pinnacle Studio 20 Ultimate is a perfectly ser...,Gets the job done ... but not as easy as it sh...,1528934400,NaN,NaN
12804,4.0,False,"04 16, 2018",A2WPL6Y08K6ZQH,B01HAP47PQ,{'Platform:': ' PC Disc'},Narut Ujnat,A program that is fairly easy to use and provi...,Good overall program.,1523836800,NaN,NaN


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12805 entries, 0 to 12804
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         12805 non-null  float64
 1   verified        12805 non-null  bool   
 2   reviewTime      12805 non-null  object 
 3   reviewerID      12805 non-null  object 
 4   asin            12805 non-null  object 
 5   style           7161 non-null   object 
 6   reviewerName    12796 non-null  object 
 7   reviewText      12804 non-null  object 
 8   summary         12799 non-null  object 
 9   unixReviewTime  12805 non-null  int64  
 10  vote            3902 non-null   object 
 11  image           71 non-null     object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 1.2+ MB


In [14]:
df['reviewerID'].nunique()

1826

In [15]:
df['asin'].nunique()

802